# Semantic Similarity

## Steps
1. sentence -> PhoBERT Embedding -> sentence embedding
2. sentence embedding cosine similarity

3. word2vec

## Fields
1. problem - solution - domain - project_name
2. customer behavior - problem
3. outstands - solution

In [ ]:
!pip3 install torch transformers

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
line = "Tôi là sinh_viên trường đại_học Công_nghệ ."

input_ids = torch.tensor([tokenizer.encode(line)])

with torch.no_grad():
  features = phobert(input_ids) # Models outputs are now tuples

print(input_ids)

In [ ]:
from underthesea import word_tokenize

max_len=100

sentence="Chào bạn, mình là Nghi sinh viên trường Đại học Bách khoa."

# Word Segmented Sentence
sentence = word_tokenize(sentence, format="text")

# Tokenize: Padding -> Pytorch Tensor
# https://huggingface.co/transformers/v3.3.1/internal/tokenization_utils.html
sentence_tokenizer = tokenizer(text=sentence,padding='max_length', max_length=max_len) # tf, np, pt
input_ids = torch.tensor([sentence_tokenizer.input_ids])
attention_mask = sentence_tokenizer.attention_mask
print(input_ids)


In [ ]:
import numpy

# Output: the hidden state vector of pre-defined hidden size corresponding to each token in the input sequence.
encoded_sequence = phobert_padding_tokenizer(sentence)
input_ids = encoded_sequence['input_ids']
attention_mask = encoded_sequence['attention_mask']

doc = """Đây là cô gái đến từ hôm qua"""
phobert_sentence_embedding(doc)

## Word2Vec
[Github Word2Vec Vietnamese](https://github.com/sonvx/word2vecVN?fbclid=IwAR3JA6FwBTSotl6u_JkXBuHmaeTGRTmkWSo_zCjdqp0zArK2mUJ2tc15dvU)

In [ ]:
!pip3 install gensim pot #pot is needed

In [ ]:
from gensim.models import KeyedVectors
from gensim import models

word2vec_path='word2vec/baomoi.model.bin'
word_vectors = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
word_vectors["văn_bản"]

In [ ]:
#https://radimrehurek.com/gensim/models/keyedvectors.html

from underthesea import word_tokenize

docs = word_tokenize("khách sạn du lịch máy bay đặt vé nghỉ dưỡng thư giãn")
print(word_vectors.doesnt_match(docs)) #đặt

# sim = word_vectors.most_similar('yêu', topn=50)
# print(sim)

w1 = "dịch_vụ"
w2 = "khách_sạn"
result = word_vectors.similarity(w1, w2)
print(f'<{w1}> is {result} similar to <{w2}>')

# most_similar_key, similarity = result[0]  # look at the first match
# print(f"{most_similar_key}: {similarity:.4f}")

sentence_obama = word_tokenize('Obama là tổng thống da màu nỗi tiếng được phỏng vấn', format="text").lower().split()
sentence_president = 'Ngài Tổng thống chào báo chí ở Chicago'
sentence_president = word_tokenize(sentence_president, format="text").lower().split()

#Compute the Word Mover’s Distance between two documents. 
# similarity = word_vectors.wmdistance(sentence_obama, sentence_president)
# print(f"{sentence_obama}\n {sentence_president} \n -> {similarity:.4f}")


similarity = word_vectors.n_similarity(['cháo', 'li'], ['nước', 'đồ_ăn'])
print(f"n_sim -> {similarity:.4f}")


# vector = word_vectors['chén']  # numpy vector of a word
# vector.shape

# vector = word_vectors.get_vector('văn', norm=True)
# vector.shape

In [ ]:
def keyword_extractor(sentence):
  sent_embedding = phobert_sentence_embedding(sentence)
  word_vectors()

In [ ]:
def is_document_related(sent1, sent2, word2vec):
  return True

In [ ]:
doc1='Nhờ các công cụ đa phương tiện của máy tính như văn bản, đồ họa, hình ảnh, âm thanh, giáo viên sẽ xây dựng được bài giảng sinh động thu hút sự tập trung của người học'
doc2='Ứng dụng công nghệ thông tin vào dạy học.'

is_document_related(doc1, doc2, word_vectors)


# SIMILARITY AND PENALTY

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer
from underthesea import word_tokenize

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

In [3]:
def phobert_padding_tokenizer(sentence, max_len=100):
    # Word Segmented Sentence
    seg_text = word_tokenize(sentence, format="text")
    
    encoding = tokenizer.encode_plus(
                seg_text,
                truncation=True,
                add_special_tokens=True,
                max_length=max_len,
                padding='max_length',
                return_attention_mask=True,
                return_token_type_ids=False,
                return_tensors='pt',
            )
            
    return {
        'text': seg_text,
        'input_ids': encoding['input_ids'], #torch.Size([1, 100])
        'attention_mask': encoding['attention_mask'], #torch.Size([1, 100])
        # 'targets': torch.tensor(label, dtype=torch.long),
    }

In [4]:
def phobert_sentence_embedding(sentence, max_len=100):
    # Word Segmented Sentence
    seg_text = word_tokenize(sentence, format="text")
    input_ids = torch.tensor([tokenizer.encode(seg_text)])

    #disables gradient calculation.
    with torch.no_grad():
        features = phobert(input_ids).last_hidden_state[:,0,:] # Models outputs are now tuples

    print(features.shape)
    # torch.Size([1, 768])
    return features

In [5]:
def phobert_doc_similarity(sent1, sent2):
  embedded_sent1 = phobert_sentence_embedding(sent1)
  embedded_sent2 = phobert_sentence_embedding(sent2)

# SimeCSE_Vietnamese
[ref](https://huggingface.co/VoVanPhuc/sup-SimCSE-VietNamese-phobert-base)

In [2]:
!pip3 install -U sentence-transformers pyvi plotly nbformat

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pyvi import ViTokenizer, ViPosTagger

ViTokenizer.tokenize(u"Trường đại học bách khoa hà nội")

ViPosTagger.postagging(ViTokenizer.tokenize(u"Trường đại học Bách Khoa Hà Nội"))

from pyvi import ViUtils
ViUtils.remove_accents(u"Trường đại học bách khoa hà nội")

from pyvi import ViUtils
ViUtils.add_accents(u'truong dai hoc bach khoa ha noi')

'Trường Đại học Bách Khoa hà nội'

In [4]:
import torch
from transformers import AutoModel, AutoTokenizer
from pyvi.ViTokenizer import tokenize as pyvi_tokenizer

PhobertTokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
SimPhobertModel = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")

# vie_phobert_model = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')


In [1]:
sentences = ['Kẻ đánh bom đinh tồi tệ nhất nước Anh. Người đàn ông đó là kẻ thối tha',
          'Nghệ sĩ làm thiện nguyện - minh bạch là việc cấp thiết.',
          'Bắc Giang tăng khả năng điều trị và xét nghiệm.',
          'HLV futsal Việt Nam tiết lộ lý do hạ Lebanon.',
          'việc quan trọng khi kêu gọi quyên góp từ thiện là phải minh bạch, giải ngân kịp thời.',
          '20% bệnh nhân Covid-19 có thể nhanh chóng trở nặng.',
          'Thái Lan thua giao hữu trước vòng loại World Cup.',
          'Cựu tuyển thủ Nguyễn Bảo Quân: May mắn ủng hộ futsal Việt Nam',
          'Chủ ki-ốt bị đâm chết trong chợ đầu mối lớn nhất Thanh Hoá.',
          'Bắn chết người trong cuộc rượt đuổi trên sông.'
          ]

sentences_tokenizer = [pyvi_tokenizer(sentence) for sentence in sentences]

inputs = PhobertTokenizer(sentences_tokenizer, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    embeddings = SimPhobertModel(**inputs, output_hidden_states=True, return_dict=True).pooler_output

NameError: name 'pyvi_tokenizer' is not defined

In [169]:
embeddings.shape

torch.Size([10, 768])

In [171]:
def document_embeddings(documents):
  documents_tokenizer = [pyvi_tokenizer(docu) for docu in documents]

  inputs = PhobertTokenizer(documents_tokenizer, padding=True, truncation=True, return_tensors="pt")
  print(inputs)

  with torch.no_grad():
    embeddings = SimPhobertModel(**inputs, output_hidden_states=True, return_dict=True).pooler_output 

  return embeddings

In [5]:
document_tokenizer = pyvi_tokenizer("Chào bạn")
document_tokenizer
# inputs = PhobertTokenizer(document_tokenizer, padding=True, truncation=True, return_tensors="pt")
# inputs

# with torch.no_grad():
#   embeddings = SimPhobertModel(**inputs, output_hidden_states=True, return_dict=True).pooler_output

# embeddings.shape

'Chào bạn'

### KMean Clustering
1. number of clusters
2. center points
[Key Parameters](https://machinelearningcoban.com/2017/01/01/kmeans/#cac-ham-so-can-thiet-cho-k-means-clustering)

In [10]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import pairwise_distances

def cosine_clustering(embeddings, num_clusters):
    cosine_similarities = 1 - pairwise_distances(embeddings, metric='cosine')
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(cosine_similarities)
    return kmeans.labels_

In [11]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import pairwise_distances

def euclide_clustering(embeddings, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(embeddings)
    return kmeans.labels_

### Visualize Clustering

In [12]:
def visualization(sentences, embeddings, clustering, num_clusters = 5):
    X = np.array(embeddings)
    pca = PCA(n_components=3)
    result = pca.fit_transform(X)
    df = pd.DataFrame({
    'sent': sentences,
    'cluster': clustering(embeddings, num_clusters).astype(str),
    'x': result[:, 0],
    'y': result[:, 1],
    'z': result[:, 2]
    })
    fig = px.scatter_3d(df, x='x', y='y', z='z',
              color='cluster', hover_name='sent',
              range_x = [df.x.min()-1, df.x.max()+1],
              range_y = [df.y.min()-1, df.y.max()+1],
              range_z = [df.z.min()-1, df.z.max()+1])

    fig.update_traces(hovertemplate= '<b>%{hovertext}</b>')
    fig.show()

In [13]:
example_documents = [
'Một nền tảng quảng cáo dựa trên dữ liệu tập hợp tất cả dữ liệu khách hàng của công ty và giúp người quảng cáo có thể truy cập được ngay từ đầu ngón tay. Một cách bài trừ. Một tổ hợp SaaS.',
'Một nền tảng cho các nhà phát triển để xây dựng trò chơi trên dữ liệu do người dùng tạo ra. Ban đầu, hệ thống tập trung vào việc giúp mọi người đưa dữ liệu của người dùng vào trò chơi - đặc biệt là thống kê thể thao - và để mọi người chơi trò chơi.',
'Một công ty phần cứng sản xuất một loạt các sản phẩm IoT để giám sát dữ liệu cảm biến, kết nối với đám mây, và biến dữ liệu đó thành các phân tích. Nếu phải làm.',
' Một công ty công nghệ tài chính tạo ra công nghệ giúp các doanh nghiệp hoạt động hàng ngày. Họ muốn trở thành công ty có thể hiểu được tất cả dữ liệu mà các công ty đã có trong hệ thống của họ.',
'Một startup cho phép doanh nghiệp được trả tiền cho tất cả hành động và dữ liệu của người dùng. Và thế là chúng ta có những dụng cụ.',
'Công ty đang phát triển một phương pháp mới để đánh giá các nhà phát triển phần mềm, được gọi là "kỹ thuật số hồ sơ", sử dụng một thuật toán lấy dữ liệu thời gian thực từ nguồn cấp dữ liệu truyền thông xã hội của nhà phát triển để đo lường kỹ năng phát triển phần mềm của họ.',
'Một nền tảng trí tuệ dữ liệu cho phép khách hàng "nhìn thấy dữ liệu của họ trong nháy mắt" thông qua các biểu đồ và biểu đồ.',
'Một sản phẩm SaaS giúp mọi người dễ dàng đảm bảo tất cả dữ liệu của họ được an toàn và nó không rơi vào tay kẻ xấu.',
'Một sản phẩm giúp các tổ chức tìm ra tài năng tốt nhất với cơ sở dữ liệu hợp tác của những người tìm việc và các ứng viên.',
'Một nền tảng tuyển dụng có AI giúp các công ty tìm đúng tài năng cho một công việc bằng cách phân tích các bài đăng trên mạng xã hội của ứng viên và các dữ liệu khác.',
'Một ứng dụng SaaS để quản lý và thu thập các khoản thanh toán, dữ liệu và các tài sản khác.',
'Một startup kết hợp các đoạn phim và âm thanh về lời khai tại toà với dữ liệu để tạo ra một câu chuyện mạnh mẽ cho khán giả tại toà.',
'Một công cụ để thiết kế các hệ thống lưu trữ dữ liệu, như hệ thống lưu trữ đám mây. Công ty khởi nghiệp này giúp người dùng tạo ra các hệ thống lưu trữ đám mây từ đầu',
'Một nền tảng giúp các công ty ở châu Âu và Mỹ Latinh quản lý chi phí hiệu quả hơn. Nền tảng được xây dựng để giúp các công ty theo dõi và quản lý chi phí của họ, và công ty nhằm mục đích cung cấp dữ liệu đó cho các doanh nghiệp khác.',
]


In [14]:
example_documents = ['Flock Safety là hệ thống an toàn công cộng đầu tiên cho phép các cộng đồng tư nhân và cơ quan thực thi pháp luật cùng nhau làm việc để loại bỏ tội phạm. Chúng tôi cam kết bảo vệ quyền riêng tư của con người và giảm thiểu thiên kiến trong việc áp dụng luật pháp với sự phát triển của công nghệ tốt nhất trong nhóm bắt nguồn từ thiết kế đạo đức, giúp kết hợp dân thường và công chức theo đuổi một xã hội an toàn hơn, công bằng hơn. Phương pháp An toàn như một dịch vụ của chúng tôi bao gồm các thiết bị giá cả phải chăng chạy bằng LTE và năng lượng mặt trời có thể được lắp đặt ở bất cứ đâu. Công nghệ của chúng tôi phát hiện và nắm bắt các chi tiết khách quan, giải mã bằng chứng trong thời gian thực và đưa các dẫn liệu điều tra vào tay những người quan trọng. Trong khi an toàn là một ngành kinh doanh nghiêm túc, chúng tôi là một đội ngũ hỗ trợ tối ưu hóa các trải nghiệm từ xa để tạo ra các mối quan hệ mạnh mẽ và vui vẻ ngay cả khi chúng tôi cách xa nhau về mặt thể chất. Đội ngũ nhân viên chăm chỉ của chúng tôi phát triển mạnh mẽ trong một môi trường tích cực và hòa nhập, nơi thiên kiến hành động được tưởng thưởng.',
 'Khởi nghiệp này nhằm mục đích giúp mọi người cá nhân hoá, theo dõi và chia sẻ tất cả dữ liệu trên xe hơi và các phương tiện khác của họ.',
 'Một nền tảng phân tích cho các nhà bán lẻ lớn cho phép họ theo dõi dữ liệu bán hàng trên các kênh và sử dụng dữ liệu đó để nhanh chóng cải thiện trải nghiệm của khách hàng.',
 'Một startup sẽ giúp các cửa hàng tạp hóa xếp hạng sản phẩm của họ bằng cách sử dụng các tín hiệu từ người tiêu dùng, chẳng hạn như liệu họ có lịch sử mua một số mặt hàng cụ thể hay không. Công ty khởi nghiệp sẽ sử dụng dữ liệu từ các nhà bán lẻ như Walmart, Amazon và Target để phát triển cách xác định mặt hàng tốt nhất để bán.',
 'Một nền tảng phần mềm giúp các nhà bán lẻ quản lý hàng tồn kho của họ.',
 'Công ty khởi nghiệp này đang tìm cách số hóa quy trình nhập dữ liệu văn phòng hậu cần. Có trụ sở tại San Francisco, công ty khởi nghiệp này hiện đang tập trung vào ngành bảo hiểm, mặc dù nó đang tìm cách mở rộng sang thị trường bán lẻ và khách sạn.',
 'Một cách mới để các công ty bảo hiểm quản lý dữ liệu sức khỏe của khách hàng và đưa ra quyết định chính xác hơn về việc điều trị bao gồm những gì.',
 'Một công ty khởi nghiệp bảo hiểm tại gia với mục tiêu cung cấp dịch vụ khách hàng tốt hơn và giảm chi phí bằng cách thu thập dữ liệu y tế ít tốn kém hơn. Công ty cũng có một dịch vụ giúp họ có thể lập một kế hoạch chăm sóc sức khỏe tùy chỉnh cho bạn.',
 'Ý tưởng là xây dựng một nền tảng để tự động hóa quá trình yêu cầu bồi thường bảo hiểm phức tạp bằng cách tự động hóa công việc thu thập và phân tích dữ liệu tẻ nhạt và dễ bị lỗi. Nó nhằm giúp các công ty bảo hiểm, môi giới bảo hiểm và hệ thống bệnh viện.',
 'Một công ty khởi nghiệp tập trung vào bảo hiểm thu thập dữ liệu từ các tài xế sử dụng điện thoại Android của họ và sau đó sử dụng dữ liệu để bán bảo hiểm cho họ. Đó là một công ty bảo hiểm không thực sự viết các hợp đồng bảo hiểm.',
 'Công ty khởi nghiệp này giúp các công ty bảo hiểm quản lý dữ liệu của họ và giúp việc giải quyết khiếu nại dễ dàng hơn. Họ đã xây dựng một cơ sở dữ liệu về khiếu nại y tế, mà Insurify đưa vào hệ thống của họ để giúp các công ty bảo hiểm dễ dàng giải quyết khiếu nại hiệu quả hơn.',
 'Một công ty bảo mật và an ninh mạng giúp các công ty đảm bảo rằng dữ liệu của họ được giữ an toàn.',
 'Phòng chống Youtube hacker: phát triển một hệ thống nhận dạng và xác thực nâng cao sử dụng công nghệ nhận dạng sinh trắc học, như nhận dạng khuôn mặt hoặc dấu vân tay, để bảo vệ quyền sở hữu và quản lý kênh YouTube của người dùng. Hệ thống sẽ yêu cầu xác thực sinh trắc học từ chủ sở hữu kênh trước khi thực hiện bất kỳ thay đổi quan trọng nào trên kênh, như thay đổi thông tin cơ bản, tải lên video mới hoặc thay đổi cài đặt quản lý. Ngoài ra, hệ thống cũng có thể sử dụng công nghệ trí tuệ nhân tạo để theo dõi hoạt động bất thường và cảnh báo cho chủ sở hữu khi phát hiện bất kỳ hoạt động nghi ngờ nào liên quan đến việc truy cập hoặc thay đổi kênh. Điều này giúp người dùng bảo vệ tài sản trực tuyến của họ một cách hiệu quả và đảm bảo tính bảo mật cho kênh YouTube của mình.',
 'Một công ty khởi nghiệp về bảo mật muốn các doanh nghiệp dễ dàng nhận biết khi nào nhân viên của họ truy cập dữ liệu nhạy cảm hoặc chia sẻ nó với nhầm người.',
 'Một startup bảo mật cung cấp một ứng dụng cho phép người dùng khoá dữ liệu của họ và thực hiện xóa từ xa.',
 'Công ty tin vào một mô hình bảo vệ quyền riêng tư để lưu trữ và chia sẻ dữ liệu người dùng để tăng thêm giá trị cho người dùng của họ và đang xây dựng một nền tảng để cho phép người dùng kiểm soát cách sử dụng dữ liệu của họ.',
 'Một công ty đang xây dựng một cách mới để theo dõi và tổ chức dữ liệu trong mạng của một công ty. Nó hoạt động bằng cách xác định cơ sở hạ tầng, giám sát nó và gửi cảnh báo khi phát hiện thay đổi.',
 'Một nền tảng toàn cầu giúp tạo điều kiện thuận lợi cho việc bán dữ liệu người dùng trên web đen. Khách hàng của công ty bao gồm hàng trăm thương hiệu lớn nhất thế giới.',
 'Một công cụ giúp "nhận dạng dữ liệu", cho phép người dùng tìm kiếm thông qua các tệp và hình ảnh trong máy tính hoặc thiết bị di động của họ để đảm bảo họ không vô tình xoá hoặc xoá bất cứ thứ gì',
 'Một công ty giúp các tổ chức tuân thủ Quy định bảo vệ dữ liệu chung của Liên minh châu Âu (GDPR).',
 'Một công ty muốn đảm bảo dữ liệu của bạn được sử dụng vì những lý do chính đáng. Họ đang xây dựng phần mềm và phần cứng cho phép bạn biết dữ liệu của bạn đang được bán hay sử dụng.',
 'Không ai muốn chuyển dữ liệu cá nhân của mình sang một nhà cung cấp dịch vụ đám mây mới. Thay vào đó, họ muốn giữ dữ liệu hiện có của mình trong đám mây, nhưng có được sự yên tâm từ một cơ sở hạ tầng đa thuê bao cấp doanh nghiệp. Công ty khởi nghiệp này muốn giúp các doanh nghiệp làm điều đó với nền tảng phần mềm dưới dạng dịch vụ, chăm sóc cơ sở hạ tầng để họ có thể tập trung vào sản phẩm của mình.',
 'Một cách an toàn hơn để chia sẻ dữ liệu nhạy cảm với các công ty và đối tác khác trong một kịch bản IoT',
 'Ý tưởng: Một dịch vụ sao lưu trực tuyến sử dụng mã hóa để bảo vệ dữ liệu khỏi bất kỳ ai ngoài chủ sở hữu của một tài khoản cụ thể.',
 'Một công ty khởi nghiệp kết nối các công ty với các giải pháp cấp doanh nghiệp để quản lý quyền riêng tư và tuân thủ dữ liệu.',
 'Một công ty giúp các công ty B2B hiểu được dữ liệu của họ. Họ cũng đang trong quá trình xây dựng một công cụ để các công ty tạo ra một đội ngũ B2B.',
 'Một công ty khởi nghiệp từ Boston với một nhóm 3 người giúp các công ty xây dựng các mô hình rủi ro tùy chỉnh cho các thị trường cụ thể, sau đó xây dựng các mô hình tự động hóa quá trình thu thập và phân tích dữ liệu ở quy mô.',
 'Một giải pháp kế toán dựa trên đám mây dành cho các chủ doanh nghiệp nhỏ. Nền tảng SaaS của công ty sẽ kết nối hàng triệu doanh nghiệp nhỏ với tài khoản của họ và cung cấp một "trung tâm" cho dữ liệu tài chính của họ.',
 'Chúng tôi muốn phát triển một công ty giúp doanh nghiệp hiệu quả hơn bằng cách giúp họ theo dõi, tổ chức và đưa ra quyết định tốt hơn xung quanh dữ liệu của họ.']

In [172]:
embeddings = document_embeddings(example_documents)
# visualization(example_documents, embeddings, cosine_clustering)
# visualization(example_documents, embeddings, euclide_clustering)
embeddings
# embeddings[2].shape

{'input_ids': tensor([[    0, 14632,  6296,  ...,  1740,     5,     2],
        [    0, 18946,    23,  ...,     1,     1,     1],
        [    0,   242,  2170,  ...,     1,     1,     1],
        ...,
        [    0,   242,   263,  ...,     1,     1,     1],
        [    0,   242,   722,  ...,     1,     1,     1],
        [    0,   746,   202,  ...,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


tensor([[ 0.1651,  0.2917,  0.0148,  ..., -0.1396,  0.1996,  0.3298],
        [-0.0313,  0.0524, -0.1016,  ...,  0.0623,  0.4149,  0.3207],
        [ 0.0459,  0.0392, -0.2095,  ..., -0.1083, -0.0990,  0.4343],
        ...,
        [-0.0494,  0.4466, -0.2334,  ..., -0.0495,  0.2027,  0.3755],
        [-0.0564,  0.2560, -0.1161,  ..., -0.1146,  0.0517,  0.2506],
        [-0.2317, -0.0032, -0.0562,  ...,  0.0150,  0.1689,  0.0449]])

## Vector Similarity Search - Search Top K
1. KD Tree, Ball Tree -> cannot use cosine (restricted)
2. Brute
3. Approximate nearest neighbor (ANN)

### Brute Search

In [16]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def nearest_neighbors(req, documents, topk = 5):
  doc_embeddings = document_embeddings(documents)
  req_embedding = document_embeddings(req)
  nbrs = NearestNeighbors(n_neighbors=topk, algorithm='brute', metric='cosine')
  nbrs.fit(doc_embeddings)
  simi = nbrs.kneighbors(req_embedding, return_distance=False)
  
  # res = [f"{documents[idx]}" for idx in simi.flatten()[::-1]]
  # print(res)
  print(simi)


In [40]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_search(query, documents, topk):
  doc_embeddings = document_embeddings(documents)
  query_embedding = document_embeddings(query)
  print(query_embedding.shape, doc_embeddings[0].shape)
  cos_similarities = cosine_similarity(doc_embeddings, query_embedding)

  # Get indices of nearest neighbors based on cosine similarity
  indices = np.argsort(cos_similarities.flatten())[::-1][:topk]
  # print(indices)
  # Print nearest neighbors
  print("\nNearest neighbors:")
  # for i, idx in enumerate(indices):
  #   print(f"Vector {i + 1}: {documents[idx]}, Cosine Similarity: {cos_similarities[idx][0]}")
  res = [documents[idx] for idx in indices]
  # print(res)

  return res


In [41]:
requirement = ["Phần mềm SaaS lưu trữ dữ liệu"]
# nearest_neighbors(requirement, example_documents, 10)
cosine_similarity_search(requirement, example_documents, 10)

torch.Size([1, 768]) torch.Size([768])

Nearest neighbors:


['Một công ty bảo mật và an ninh mạng giúp các công ty đảm bảo rằng dữ liệu của họ được giữ an toàn.',
 'Một giải pháp kế toán dựa trên đám mây dành cho các chủ doanh nghiệp nhỏ. Nền tảng SaaS của công ty sẽ kết nối hàng triệu doanh nghiệp nhỏ với tài khoản của họ và cung cấp một "trung tâm" cho dữ liệu tài chính của họ.',
 'Một startup bảo mật cung cấp một ứng dụng cho phép người dùng khoá dữ liệu của họ và thực hiện xóa từ xa.',
 'Không ai muốn chuyển dữ liệu cá nhân của mình sang một nhà cung cấp dịch vụ đám mây mới. Thay vào đó, họ muốn giữ dữ liệu hiện có của mình trong đám mây, nhưng có được sự yên tâm từ một cơ sở hạ tầng đa thuê bao cấp doanh nghiệp. Công ty khởi nghiệp này muốn giúp các doanh nghiệp làm điều đó với nền tảng phần mềm dưới dạng dịch vụ, chăm sóc cơ sở hạ tầng để họ có thể tập trung vào sản phẩm của mình.',
 'Công ty tin vào một mô hình bảo vệ quyền riêng tư để lưu trữ và chia sẻ dữ liệu người dùng để tăng thêm giá trị cho người dùng của họ và đang xây dựng một n

In [ ]:
with open('../data/vietnamese-stopwords.txt', encoding='utf8') as f:
    for line in f:
        stop_words.append(line.strip())

## Data Preprocessing

In [ ]:
def read_txt(filename):
  with open(filename, 'r') as f:
    lines = f.readlines()
    lines_remove_newline = [line[:-1] for line in lines]

  return lines_remove_newline

In [ ]:
# FOLDER CONSTANT PATH

data_folder = '../data/'
sub_folder_similar = data_folder + 'similar'
sub_folder_vie = data_folder + 'vie'

In [ ]:
import pandas as pd

# FOLDER CONSTANT PATH

data_folder = '../data/'
sub_folder_similar = data_folder + 'similar/'
sub_folder_vie = data_folder + 'vie/'
file_unpreproc = sub_folder_similar + 'unpreproc.txt'
file_valid_idea = sub_folder_vie + 'valid_idea.csv'
file_new_dataset = sub_folder_similar + 'dataset_new.txt'

unpreproc_list = read_txt(file_unpreproc)
unpreproc_df = pd.DataFrame(unpreproc_list, columns=['sentence'])
unpreproc_track_df = pd.DataFrame(unpreproc_list, columns=['sentence'])
valid_df = pd.read_csv(file_valid_idea, delimiter=';', usecols=['sentence', 'spam'])
result = {'sentence': [], 'field': []} 


# unpreproc_df
# print(unpreproc_list)

for sent_field in unpreproc_df['sentence']:
  for valid_sent in valid_df['sentence']:
    try:
      sent, field = sent_field.split(' - ')
    except:
      print(sent_field)
    if sent[:-15] in valid_sent:
      if valid_sent not in result['sentence']:
        result['sentence'] += [valid_sent]
        result['field'] += [field]
        unpreproc_track_df = unpreproc_track_df[unpreproc_track_df['sentence'] != sent_field]

result_df_0 = pd.DataFrame(result, columns=['sentence', 'field'])
result_df_0

In [ ]:
import pandas as pd

# FOLDER CONSTANT PATH

data_folder = '../data/'
sub_folder_similar = data_folder + 'similar/'
sub_folder_vie = data_folder + 'vie/'
# file_unpreproc = sub_folder_similar + 'unpreproc.txt'
# file_valid_idea = sub_folder_vie + 'valid_idea.csv'
file_new_dataset = sub_folder_similar + 'dataset_new.txt'

sent_field_dataset = read_txt(file_new_dataset)
result = {'sentence': [], 'field': []} 
for sent_field in sent_field_dataset:
    try:
      sent, field = sent_field.split(' - ')
      if sent not in result['sentence']:
        result['sentence'] += [sent]
        result['field'] += [field]
    except:
      print("Except >> ", sent_field)

result_df_1 = pd.DataFrame(result, columns=['sentence', 'field'])
result_df_1

In [170]:
diction = {'sent': {}, 'abc': {}, 'cde': {} }
df = pd.DataFrame(diction)

In [ ]:
# FOLDER CONSTANT PATH

data_folder = '../data/'
sub_folder_similar = data_folder + 'similar/'
sub_folder_vie = data_folder + 'vie/'
file_olabel = sub_folder_similar + 'old_label.txt'
file_ovalid_idea = sub_folder_similar + 'old_valid_idea.csv'

old_valid_df = pd.read_csv(file_ovalid_idea, delimiter=';', usecols=['sentence', 'spam'])
olabel_list = read_txt(file_olabel)
olabel_df = pd.DataFrame(olabel_list, columns=['field'])

result_df_2 = pd.concat([old_valid_df['sentence'], olabel_df], axis=1)
result_df_2

In [ ]:
# FOLDER CONSTANT PATH

data_folder = '../data/'
sub_folder_similar = data_folder + 'similar/'
sub_folder_vie = data_folder + 'vie/'
classified = sub_folder_similar + 'classified.txt'

classified_list = read_txt(classified)
result = {'sentence': [], 'field': []}
for sent in classified_list:
  try:
    sentence, field = sent.split(' - ')
    result['sentence'] += [sentence]
    result['field'] += [field]
  except:
    print(sent)

result_df_3 = pd.DataFrame(result, columns=['sentence', 'field'])
result_df_3

In [ ]:
result = pd.concat([result_df_0, result_df_2, result_df_1, result_df_3], ignore_index=True)
result

In [ ]:
# FOLDER CONSTANT PATH

data_folder = '../data/'
sub_folder_similar = data_folder + 'similar/'
sub_folder_vie = data_folder + 'vie/'
file_valid_idea = sub_folder_vie + 'valid_idea.csv'
unfinished_csv = sub_folder_similar + 'unfinished.csv'
# file_new_dataset = sub_folder_similar + 'dataset_new.txt'


valid_df = pd.read_csv(file_valid_idea, delimiter=';', usecols=['sentence', 'spam'])
temp = pd.read_csv(file_valid_idea, delimiter=';', usecols=['sentence', 'spam'])
classified_sent_df = pd.concat([result_df_0, result_df_2, result_df_1, result_df_3, result_df_5], ignore_index=True)

for valid_sent in valid_df['sentence']:
  for classified_sent in classified_sent_df['sentence']:
    if classified_sent[:-10] in valid_sent:
      idx = valid_df[valid_df.sentence == valid_sent].index
      print(">> ", classified_sent[:-10])
      print("--- ", valid_sent)
      print("Index = ", idx.values.tolist())
      valid_df = valid_df.drop(idx.values.tolist())

valid_df.to_csv(unfinished_csv, index=False, sep=';')

In [ ]:
import pandas as pd

# FOLDER CONSTANT PATH

data_folder = '../data/'
sub_folder_similar = data_folder + 'similar/'
sub_folder_vie = data_folder + 'vie/'
file_unpreproc = sub_folder_similar + 'dataset_unfi.txt'
file_valid_idea = sub_folder_similar + 'unfinished.csv'
file_new_dataset = sub_folder_similar + 'dataset_new.txt'
unfinished2_csv = sub_folder_similar + 'unfinished2.csv'


unpreproc_list = read_txt(file_unpreproc)
unpreproc_df = pd.DataFrame(unpreproc_list, columns=['sentence'])
unpreproc_track_df = pd.DataFrame(unpreproc_list, columns=['sentence'])
valid_df = pd.read_csv(file_valid_idea, delimiter=';', usecols=['sentence', 'spam'])
result = {'sentence': [], 'field': []} 


# unpreproc_df
# print(unpreproc_list)

for sent_field in unpreproc_df['sentence']:
  for valid_sent in valid_df['sentence']:
    try:
      sent, field = sent_field.split(' - ')
    except:
      print(sent_field)
    if sent[5:36] in valid_sent:
      if valid_sent not in result['sentence']:
        result['sentence'] += [valid_sent]
        result['field'] += [field]
        valid_df = valid_df[valid_df['sentence'] != valid_sent]

result_df_4 = pd.DataFrame(result, columns=['sentence', 'field'])
valid_df.to_csv(unfinished2_csv, index=False, sep=';')

In [ ]:
data_folder = '../data/'
sub_folder_similar = data_folder + 'similar/'
sub_folder_vie = data_folder + 'vie/'
file_unpreproc = sub_folder_similar + 'dataset_unfi.txt'
file_valid_idea = sub_folder_similar + 'unfinished.csv'
file_new_dataset = sub_folder_similar + 'dataset_new.txt'
classified = sub_folder_similar + 'classified.csv'
unfinished2_csv = sub_folder_similar + 'unfinished2.csv'

unpreproc_list = read_txt(file_unpreproc)
matching = {'sentence': [], 'field': []}
for sent_field in unpreproc_list:
  try:
    sent, field = sent_field.split('>')
    if sent not in matching['sentence']:
      matching['sentence'] += [sent]
      matching['field'] += [field]

  except:
    print(sent_field)

result_df_5 = pd.DataFrame(matching, columns=['sentence', 'field'])
classified_sent_df = pd.concat([result_df_0, result_df_2, result_df_1, result_df_3, result_df_5], ignore_index=True)
classified_sent_df
classified_sent_df.to_csv(classified, sep='>', index=False)

In [ ]:
classified_sent_df = classified_sent_df.sort_values('field')
classified_sent_df.to_csv(classified, sep='>', index=False)

### Requirement Dataset

In [82]:
data_folder = '../data/'
sub_folder_similar = data_folder + 'similar/'
sub_folder_vie = data_folder + 'vie/'
sub_folder_rank = data_folder + 'rank/'
file_unpreproc = sub_folder_similar + 'dataset_unfi.txt'
file_valid_idea = sub_folder_similar + 'unfinished.csv'
file_new_dataset = sub_folder_similar + 'dataset_new.txt'
classified_csv = sub_folder_similar + 'ordered_classified.csv' ## delimiter = '>'

In [ ]:
classified_df = pd.read_csv(classified_csv, delimiter='>', usecols=['sentence', 'field'])
classified_sentence_list = classified_df['sentence'].to_list()
classified_sentence_list[:10]

In [ ]:
documents = classified_sentence_list
embeddings = document_embeddings(documents)
# visualization(example_documents, embeddings, cosine_clustering)
visualization(documents, embeddings, euclide_clustering, 8)

# Tạo Requirement quote

In [19]:
data_folder = '../data/'
sub_folder_similar = data_folder + 'similar/'
sub_folder_vie = data_folder + 'vie/'
file_unpreproc = sub_folder_similar + 'dataset_unfi.txt'
file_valid_idea = sub_folder_similar + 'unfinished.csv'
file_new_dataset = sub_folder_similar + 'dataset_new.txt'
classified_csv = sub_folder_similar + 'ordered_classified.csv' ## delimiter = '>'

classified_df = pd.read_csv(classified_csv, delimiter='>', usecols=['sentence', 'field'])
classified_sentence_list = classified_df['sentence'].to_list()

In [44]:
classified_df = pd.read_csv(classified_csv, delimiter='>', usecols=['sentence', 'field'])
classified_sentence_list = classified_df['sentence'].to_list()

# idea_embedding = []
# for i, sent in enumerate(classified_sentence_list):
#   print(i)
#   embed = document_embeddings(sent)
#   idea_embedding += [embed]

In [ ]:
# for i in classified_df['sentence']:
#   print(i)

In [20]:
classified_df = pd.read_csv(classified_csv, delimiter='>', usecols=['sentence', 'field'])
classified_sentence_list = classified_df['sentence'].to_list()

# idea_embedding = []
idea_embeddings = document_embeddings(classified_sentence_list[:1000])


In [21]:
idea_embeddings_2 = document_embeddings(classified_sentence_list[1000:])

In [50]:
import torch
import os

# Assuming 'tensor_list' is your list of PyTorch tensors
# tensor_list = [torch.randn(3, 3) for _ in range(5)]

# Define a directory to save the tensors
save_dir = '../data/saved_tensors'
os.makedirs(save_dir, exist_ok=True)

In [53]:
# Save each tensor individually
def save_each_tensor(tensor_list):
    for i, tensor in enumerate(tensor_list):
        torch.save(tensor, os.path.join(save_dir, f'tensor_{i}.pt'))

    return len(tensor_list)

# Load tensors back
def load_tensor(tensor_size):
    loaded_tensors = []
    for i in range(tensor_size):
        loaded_tensors.append(torch.load(os.path.join(save_dir, f'tensor_{i}.pt')))

    return loaded_tensors

In [52]:
number_of_tensor = save_each_tensor(idea_embeddings)

In [ ]:
load_tensor(number_of_tensor)

In [65]:
type(idea_embeddings_2)

torch.Tensor

In [68]:
ideas_embed = torch.cat((idea_embeddings, idea_embeddings_2))

In [69]:
ideas_embed.shape

torch.Size([2282, 768])

In [30]:
idea_embeddings_2[0].shape
# len(idea_embeddings_2

ideas_final_embeddings = [idea_embeddings_2] + [idea_embeddings]
ideas_final_embeddings

[tensor([[ 0.0591,  0.3207, -0.2964,  ...,  0.0830,  0.2539,  0.0589],
         [-0.2171,  0.2856, -0.0538,  ..., -0.0988,  0.1220,  0.1570],
         [-0.0677,  0.2888, -0.0008,  ...,  0.0273,  0.2172,  0.3843],
         ...,
         [ 0.1161,  0.2453, -0.2541,  ..., -0.1671, -0.2207,  0.3552],
         [-0.0362,  0.4185, -0.4636,  ..., -0.0754, -0.2099,  0.0746],
         [-0.1535,  0.3608,  0.1364,  ..., -0.0968, -0.2434,  0.2811]]),
 tensor([[ 0.1651,  0.2917,  0.0148,  ..., -0.1396,  0.1996,  0.3298],
         [-0.0313,  0.0524, -0.1016,  ...,  0.0623,  0.4149,  0.3207],
         [ 0.0459,  0.0392, -0.2095,  ..., -0.1083, -0.0990,  0.4343],
         ...,
         [-0.0494,  0.4466, -0.2334,  ..., -0.0495,  0.2027,  0.3755],
         [-0.0564,  0.2560, -0.1161,  ..., -0.1146,  0.0517,  0.2506],
         [-0.2317, -0.0032, -0.0562,  ...,  0.0150,  0.1689,  0.0449]])]

In [38]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_search_2(query, embed_documents, topk):
  doc_embeddings = embed_documents
  query_embedding = document_embeddings(query)
  print(query_embedding.shape, doc_embeddings[0].shape)
  cos_similarities = cosine_similarity(doc_embeddings, query_embedding)

  # Get indices of nearest neighbors based on cosine similarity
  indices = np.argsort(cos_similarities.flatten())[::-1][:topk]
  # Print nearest neighbors
  print("\nNearest neighbors:")
  print(indices)
  # for i, idx in enumerate(indices):
  #   print(f"Vector {i + 1}: {documents[idx]}, Cosine Similarity: {cos_similarities[idx][0]}")
  # res = [documents[idx] for idx in indices]
  # print(res)

  return indices


In [64]:
def create_req_data():
  for idx, sent in enumerate(classified_df['sentence']):
    result = {}
    requirement = [sent]
    # print(">>>> ", sent)
    topk_indices = cosine_similarity_search_2(requirement, idea_embeddings, 20)
    result[idx] = topk_indices
    # for top in topk_indices:
    #   print(">>> ", classified_df['sentence'][top], '\n')
  return result

require_data = create_req_data()

torch.Size([1, 768]) torch.Size([768])

Nearest neighbors:
[  0  91  46  94  86 231  36  88 537  22  80  30 348 137 212  85  37   8
  11 226]
torch.Size([1, 768]) torch.Size([768])

Nearest neighbors:
[  1  57 890  98 587  87 365 319  58 913 407 301 137 544 727 748 839 504
 233 500]
torch.Size([1, 768]) torch.Size([768])

Nearest neighbors:
[  2 970 907 780 718 394 713 896 883 819 489 865 988 791  99 892 977 675
 654 705]
torch.Size([1, 768]) torch.Size([768])

Nearest neighbors:
[  3 394 122 937 988 387 719 623 806 118 743 713 454 677 488 698 890 995
 319 938]
torch.Size([1, 768]) torch.Size([768])

Nearest neighbors:
[  4 387 784 411 802 394   2 393 904 853 104 718   3 907 791 807 449 105
 726 463]
torch.Size([1, 768]) torch.Size([768])

Nearest neighbors:
[  5 764 153 343 812 275 520 885 608 566 179   7 233 882 513 109  31 493
 624 776]
torch.Size([1, 768]) torch.Size([768])

Nearest neighbors:
[  6 152 610 621 984 246  10   8 706 232 256 216 815 612 502   7 249 214
 634 205]
torch.

In [ ]:
def create_req_data():
  result = {}
  for idx, sent in enumerate(classified_df['sentence']):
    requirement = [sent]
    # print(">>>> ", sent)
    topk_indices = cosine_similarity_search_2(requirement, ideas_embed, 20)
    result[idx] = topk_indices
    # for top in topk_indices:
    #   print(">>> ", classified_df['sentence'][top], '\n')
  return result

full_require_topk = create_req_data()

In [73]:
full_require_topk

{0: array([   0,   91, 1848,   46,   94, 1644, 2022, 1672,   86,  231,   36,
          88, 1154, 1917,  537, 2170,   22,   80,   30,  348]),
 1: array([   1,   57,  890, 2249,   98,  587,   87,  365,  319, 1290, 1996,
          58, 1338,  913,  407,  301,  137,  544,  727, 1660]),
 2: array([   2,  970, 1325, 1370, 1706, 1300,  907, 1108, 1163, 1940, 1760,
        1953, 2205,  780, 1869,  718, 1138,  394, 1709, 1877]),
 3: array([   3, 1739,  394, 1709, 1712, 1726, 1741,  122, 1138, 1172, 1745,
        1699, 1761,  937, 1129, 1009, 1754, 1975, 1760, 1904]),
 4: array([   4,  387, 1705, 1706, 1174, 1728, 2204, 1699, 2198, 2222,  784,
        1975, 1729, 1131,  411, 2207, 1163, 1718, 1760, 1115]),
 5: array([   5,  764, 1354,  153, 2124,  343, 1393,  812,  275, 1467, 1485,
        1302,  520,  885,  608,  566,  179,    7, 2274,  233]),
 6: array([   6,  152,  610,  621,  984,  246,   10,    8,  706,  232,  256,
         216,  815,  612,  502,    7,  249,  214,  634, 1315]),
 7: array([  

In [120]:
from sklearn.metrics.pairwise import cosine_similarity
import csv

def cosine_similarity_contrastive(query, embed_documents, topk):
  doc_embeddings = embed_documents
  query_embedding = document_embeddings(query)
  cos_similarities = cosine_similarity(doc_embeddings, query_embedding)

  # Get indices of nearest neighbors based on cosine similarity
  indices = np.argsort(cos_similarities.flatten())[::-1]
  pos = indices[1:topk+1]
  neg = indices[-topk:]
  # print(pos, neg)

  return pos, neg

def write_to_csv(tuple_list):
  # Define the filename for the CSV file
  filename = '../data/rank/req_pos_neg.csv'
  # Write tuples to the CSV file
  with open(filename, 'w', newline='') as file:
      writer = csv.writer(file, delimiter='>')
      for row in tuple_list:
        print(row)
        writer.writerow(row)

req_ordered = classified_df['sentence'].to_list()

def create_bm25():
  result = []
  topk = 5
  for idx, sent in enumerate(req_ordered):
    requirement = [sent]
    # print(">>>> ", sent)
    pos, neg = cosine_similarity_contrastive(requirement, ideas_embed, topk)
    for k in range(topk):
      tuple_data = (sent, req_ordered[pos[k]], req_ordered[neg[k]])
      # print(tuple_data)
      result += [tuple_data]
  return result



In [117]:
req_pos_neg = create_bm25()

In [118]:
req_pos_neg[0]

('Flock Safety là hệ thống an toàn công cộng đầu tiên cho phép các cộng đồng tư nhân và cơ quan thực thi pháp luật cùng nhau làm việc để loại bỏ tội phạm. Chúng tôi cam kết bảo vệ quyền riêng tư của con người và giảm thiểu thiên kiến trong việc áp dụng luật pháp với sự phát triển của công nghệ tốt nhất trong nhóm bắt nguồn từ thiết kế đạo đức, giúp kết hợp dân thường và công chức theo đuổi một xã hội an toàn hơn, công bằng hơn. Phương pháp An toàn như một dịch vụ của chúng tôi bao gồm các thiết bị giá cả phải chăng chạy bằng LTE và năng lượng mặt trời có thể được lắp đặt ở bất cứ đâu. Công nghệ của chúng tôi phát hiện và nắm bắt các chi tiết khách quan, giải mã bằng chứng trong thời gian thực và đưa các dẫn liệu điều tra vào tay những người quan trọng. Trong khi an toàn là một ngành kinh doanh nghiêm túc, chúng tôi là một đội ngũ hỗ trợ tối ưu hóa các trải nghiệm từ xa để tạo ra các mối quan hệ mạnh mẽ và vui vẻ ngay cả khi chúng tôi cách xa nhau về mặt thể chất. Đội ngũ nhân viên chăm

In [ ]:
write_to_csv(req_pos_neg)

In [101]:

# Sample list of tuples
tuple_list = [
    ('John', 25, 'Engineer'),
    ('Alice', 30, 'Manager'),
    ('Bob', 28, 'Developer')
]

# Define the filename for the CSV file
filename = 'data.csv'

# Write tuples to the CSV file with a specified delimiter
delimiter = '|'  # Specify the delimiter

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file, delimiter=delimiter)
    for row in tuple_list:
        writer.writerow(row)

In [83]:
lb = [10,22,234,53,2,0,12,2]
lb[-2:]

[12, 2]

# Model

In [125]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Similarity(nn.Module):
    """
    Dot product or cosine similarity
    """
    def __init__(self, temperature):
        super().__init__()
        self.temperature= temperature
        self.cos = nn.CosineSimilarity(dim=-1)

    def forward(self, x, y):
        return self.cos(x, y) / self.temperature

class SupConLoss(nn.Module):
    def __init__(self, temperature=0.1):
        """
        Supervised Contrastive Loss was presented at https://arxiv.org/pdf/2104.08821.pdf
        """
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.sim = Similarity(temperature)
        self.cross_entropy_loss = nn.CrossEntropyLoss()
        
    def forward(self, z1, z2, z3=None):
        # compute cosine similarity between positive pairs
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))
        # Hard negative
        if z3 is not None:
            z1_z3_cos = self.sim(z1.unsqueeze(1), z3.unsqueeze(0))
            cos_sim = torch.cat([cos_sim, z1_z3_cos], 1)

        labels = torch.arange(cos_sim.size(0), device=cos_sim.device).long()
        # compute loss using cross-entropy
        loss = self.cross_entropy_loss(cos_sim, labels)
        return loss

In [156]:
from torch.optim import AdamW
import torch
from transformers import AutoModel, AutoTokenizer
from pyvi.ViTokenizer import tokenize as pyvi_tokenizer

# Khởi tạo argument
EPOCHS = 20
BATCH_SIZE = 6
ACCUMULATION_STEPS = 5
FOLD = 4
LR = 0.0001
LR_DC_STEP = 80 
LR_DC = 0.1
# CUR_DIR = os.path.dirname(os.getcwd())
# CKPT_PATH2 = 'model_ckpt2'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
FOLD = 4

phobert_tokenizer = AutoTokenizer.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
phobert_model = AutoModel.from_pretrained("VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")
loss_fn = SupConLoss(0.05)

param_optimizer = list(phobert_model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=LR)  # To reproduce BertAdam specific behavior set correct_bias=False

In [153]:
dataset_df = pd.read_csv('../data/rank/req_pos_neg.csv', sep='>')
dataset_df

,req,pos,neg
0,Flock Safety là hệ thống an toàn công cộng đầu...,Quantstamp đã bảo vệ hàng tỷ tài sản kỹ thuật ...,Một mạng xã hội phi tập trung để người dùng tr...
1,Flock Safety là hệ thống an toàn công cộng đầu...,TRM Labs là một công ty tình báo Blockchain gi...,"Vào tháng 3, chính phủ Brazil đã thông qua một..."
2,Flock Safety là hệ thống an toàn công cộng đầu...,Một cách mới để khai thác dữ liệu từ Internet ...,"Một dự án phụ cho Zenefits, một công ty phần m..."
3,Flock Safety là hệ thống an toàn công cộng đầu...,Hệ thống an ninh tinh vi không gian-mạng: Các ...,Một công ty giúp các nhà phát triển xây dựng c...
4,Flock Safety là hệ thống an toàn công cộng đầu...,Diassess Inc. đang cách mạng hóa cách phòng ng...,Cơ sở dữ liệu về doanh số hàng tháng của sản p...
...,...,...,...
11405,Newfund đang xây dựng một nền tảng sử dụng dữ ...,Một công ty khởi nghiệp sử dụng phân tích dữ l...,"Một ""Internet phi tập trung thực sự"", nơi tất ..."
11406,Newfund đang xây dựng một nền tảng sử dụng dữ ...,Một công ty khởi nghiệp cung cấp nền tảng phân...,Đây là một ứng dụng giúp người dùng theo dõi d...
11407,Newfund đang xây dựng một nền tảng sử dụng dữ ...,Công ty khởi nghiệp đang cố gắng xây dựng một ...,Sản xuất xe tải chạy bằng pin nhiên liệu hydro...
11408,Newfund đang xây dựng một nền tảng sử dụng dữ ...,"Một ""nền tảng dữ liệu lớn"" giúp các công ty đư...",Chúng ta loại bỏ CO2 và nước ra khỏi không khí...


In [157]:
def convert_df_to_list(df):
  dataset_list = []
  for index, row in df.iterrows():
    data = [row['req'], row['pos'], row['neg']]
    dataset_list += [data]

  return dataset_list

In [158]:
def each_batch_embedding(req_pos_neg_batch: list, model, tokenizer) -> tuple:
  embeddings = []
  for documents in req_pos_neg_batch:
    print('>>>  ', documents)
    documents_tokenizer = [pyvi_tokenizer(docu) for docu in documents]

    inputs = tokenizer(documents_tokenizer, padding=True, truncation=True, return_tensors="pt")

    with torch.no_grad():
      embedding = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output 
      embeddings += [embedding]

  return tuple(embeddings)

In [159]:
## embed cả batchsize hay embed từng đoạn?
batch_df = dataset_df.sample(n=BATCH_SIZE)
batch_list = convert_df_to_list(batch_df)
dataset_embedding = each_batch_embedding(batch_list, phobert_model, phobert_tokenizer)

>>>   ['Một startup cho phép doanh nghiệp được trả tiền cho tất cả hành động và dữ liệu của người dùng.', 'Một startup nơi các cá nhân có thể bán dữ liệu thu thập được từ thiết bị di động của họ với một khoản phí.', 'Tạo ra vật liệu xây dựng bền vững từ chất thải nhựa tái chế, cung cấp một giải pháp thay thế thân thiện với môi trường cho các dự án xây dựng và giảm ô nhiễm nhựa']
>>>   ['Một công cụ cho các tổ chức muốn quản lý dữ liệu của họ. Những người sáng lập cho biết họ đang làm việc với một số công ty lớn.', 'Công ty này đang xây dựng các công cụ trực quan để lưu trữ, quản lý và bảo mật các bộ dữ liệu lớn.', 'Tạo ra vật liệu xây dựng bền vững từ chất thải nhựa tái chế, cung cấp một giải pháp thay thế thân thiện với môi trường cho các dự án xây dựng và giảm ô nhiễm nhựa']
>>>   ['Công ty SaaS này giúp các công ty nhanh chóng khắc phục sự cố thời gian thực với cơ sở hạ tầng của họ, chẳng hạn như mất điện, hiệu suất web chậm hoặc dữ liệu xấu.', 'Một doanh nghiệp đưa ra giải pháp giú

In [ ]:
dataset_embedding

In [ ]:
for epoch in EPOCHS:
  phobert_model.train()
  loss = loss_fn()
  optimizer.zero_grad()